### 리스트 중복 제거

In [ ]:
from google.colab import files, drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os

#!pip install pytorch_pretrained_bert==0.4.0
os.chdir("/content/gdrive/My Drive/KorBERT")

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving df_n_v.xlsx to df_n_v (3).xlsx


In [ ]:
import pandas as pd
df = pd.read_excel("df_n_v.xlsx")

In [ ]:
df['depend']

0       지정을 발전을 위해 열정을 공직자(서기관)들이 퇴장을 앞두고 있다. 되기까지 이들은...
1       고양시민들이 출퇴근길 겪고 불편이 수 있을지 관심이 모아진다. 따르면 구간)은 4량...
2       기해년을 안양시가 창출을 종합계획을 공시했다. 따르면 종합계획은 일자리를 전략과제를...
3       의원(더불어민주당평택4)은 참석해 대해 논의했다. 일원)는 평택교육청은 증축을 통해...
4       화성시가 새해를 맞아 청사진을 공개했다. 시는 전산망을 통해 70를 종합계획을 공시...
                              ...                        
998     단행 앞두고 관심이 쏠리고 있다. 자리가 채워지기 따르면 관심이 것은 자리가 늘어나...
999     혜택을 기틀을 마련하겠습니다. 이용걸(57)회장은 의정부지회가 창립하며 안정을 위해...
1000    검찰이 귤을 훔치려다 치료감호를 청구했지만, 배심원들은 기각을 평결했다. 부장판사)...
1001    사리장마 겹쳐 우려돼 공사가 통행이 차단된다. 인천경제자유구역청은 차가 장마철을 맞...
1002    참석자들이 기념촬영을 하고 있다. 발전협의회는 토론회를 열었다. 옹진군수를 명이 참...
Name: depend, Length: 1003, dtype: object

In [ ]:
import re

In [ ]:
# 텍스트 특수부호 제거

def cleaned_text(text):
  import re
  #text = re.sub('\W+',' ',text)
  text = text.replace(' \\r','.')
  text = text.replace('\\r','.')
  text = text.replace('\\n','.')
  
  return text


def cleaned(text):
  text = cleaned_text(text)
  for i in range(2,20):
    text = text.replace('.'*i,'.')
    
  return text

def preprocess(text):
  _filter = re.compile('[^가-힣 0-9 a-z A-Z \@ \( \) \. \, \' \" \! \?]+')
  text = _filter.sub('', text)
  return text

df['content1'] = df['content'].apply(lambda x : cleaned(x))
df['content1'] = df['content1'].apply(lambda x : preprocess(x))
df['content1'] = df.content1.apply(lambda x : x.replace('.','. '))


a_ls = []
for content in df['content1']:
  clean = preprocess(content)
  a_ls.append(clean)

df['content1'] = a_ls

In [ ]:
import collections       # OrderedDict를 위해 호출
import re                # 정규표현식
import unicodedata       # 한국어 정준분해 및 문자열 확인
import six               # Python version 체크
import tensorflow as tf  # Tensorflow 파일 불러오기 및 logging
import urllib3
import json

In [ ]:
vocab_file = 'vocab.korean_morp.list'
#api_key = 'c149fccd-6639-4026-9fb8-cec175ff934c'
api_key = 'f3639d57-7635-4bd9-93b0-6eae550ab90e'

In [ ]:
def do_lang ( openapi_key, text ) :
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
	 
    requestJson = { "access_key": openapi_key, "argument": { "text": text, "analysis_code": "morp" } }
	 
    http = urllib3.PoolManager()
    response = http.request( "POST", openApiURL, headers={"Content-Type": "application/json; charset=UTF-8"}, body=json.dumps(requestJson))
    
    json_data = json.loads(response.data.decode('utf-8'))
    json_result = json_data["result"]
    
    if json_result == -1:
        json_reason = json_data["reason"]
        if "Invalid Access Key" in json_reason:
            logger.info(json_reason)
            logger.info("Please check the openapi access key.")
            sys.exit()
        return "openapi error - " + json_reason      
    else:
        json_data = json.loads(response.data.decode('utf-8'))
    
        json_return_obj = json_data["return_object"]
        
        return_result = ""
        json_sentence = json_return_obj["sentence"]
        for json_morp in json_sentence:                        
            for morp in json_morp["morp"]:
                return_result = return_result+str(morp["lemma"])+"/"+str(morp["type"])+" "

        return return_result

In [ ]:
import urllib3
import json

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1003 non-null   int64  
 1   media       1003 non-null   object 
 2   a_num       1003 non-null   int64  
 3   title       1003 non-null   object 
 4   content     1003 non-null   object 
 5   w_num       1003 non-null   int64  
 6   c_source    1003 non-null   int64  
 7   a_source    1002 non-null   float64
 8   s_num       1003 non-null   int64  
 9   v_num       1003 non-null   int64  
 10  nf_verb     1003 non-null   int64  
 11  s_verb      1003 non-null   int64  
 12  reporter    1003 non-null   int64  
 13  non_text    1003 non-null   int64  
 14  s_rate      999 non-null    float64
 15  quality     1003 non-null   int64  
 16  content1    1003 non-null   object 
 17  depend      1002 non-null   object 
dtypes: float64(2), int64(11), object(5)
memory usage: 141.2+ KB


In [ ]:
from tokenization_morp import FullTokenizer

In [ ]:
# FullTokenizer
ftk = FullTokenizer(vocab_file, do_lower_case=False)

In [ ]:
df.content1[0]

'송주현 기자 왼쪽 위부터)김운용, 이현옥, 이흥민, 노양호, 정종현, 신승일. 특례시 지정을 앞둔 105만 고양시의 발전을 위해 땀과 열정을 쏟아온 6명의 공직자(서기관)들이 아름다운 퇴장을 앞두고 있다. 경기북부의 중심도시인 고양시가 되기까지 쉼없이 달려왔던 이들은 이제 후배들에게 자리를 물려주고 인생 2막을 준비 중이다. 김운용 고양시 푸른도시사업소장은 1983년 공직에 입문해 고양시가 자연을 품은 도시가 될 수 있도록 많은 연구와 노력을 기울여왔다.  지난 28일 명예퇴직한 그는 산림보호유공을 인정받아 받은 산림청장표창을 비롯해, 국무총리와 국방부장관, 도지사 표창들이 묵묵히 업무에 충실해 온 그의 공직생활을 보여준다. 40년 세월동안 고양시를 지켜온 이현옥 교육문화국장도 공직을 마무리하기 위해 공로연수에 들어간다.  부드럽고 섬세한 그의 업무스타일은 후배들에게도 많은 신뢰와 박수를 받아왔다. 정부부처 등으로부터 표창을 13회나 받은 이흥민 민생경제국장 역시 고양시에게는 아까운 인재다.  그는 고양지역 주민들이 원활한 생활지원을 받을 수 있도록 늘 주민들과 함께하며 고양시 행정에 현장의 목소리를 담아왔다. 노양호 여성가족국장도 고양지역 발전을 위해 40년 넘게 예산법무과 등 주요부서에서 활약해 왔다.  특히 교육지원과장 시절에는 고양지역 학생들의 교육환경 개선을 위해 노력을 아끼지 않았다. 농업 분야 전문가인 정종현 농업기술센터소장도 고양지역의 농업이 활성화되기까지 많은 자취를 남겼다.  2016년에는 녹조근정훈장 영예를 받기도 했다. 신승일 시민안전주택국장도 이들과 함께 공로연수에 들어간다.  신 국장은 경기북부 중심 도시가 된 고양시의 미래설계를 담당해 온 일등 공신 인물이다.  그동안 고양시 발전을 위해서는 해당 업무를 담당하는 공직자 역시 계속적인 발전이 필요하다고 강조해 온 그는 2011년에는 경희대에서 경영학을 전공하고 2014년에는 고려대에서 건축공학과 석사과정을 마쳤다.  풍부한 학식과 오랜 행정경험을 통해 쌓은 노하우, 업무를 추진하는 

In [ ]:
print(ftk.tokenize(do_lang(api_key,df.content1[0])))

['송', '주', '현/NNP_', '기자/NNG_', '왼쪽/NNG_', '위/NNG_', '부터/JX_', ')/SS_', '김', '운', '용/NNP_', ',/SP_', '이현', '옥/NNP_', ',/SP_', '이', '흥', '민/NNP_', ',/SP_', '노', '양', '호/NNP_', ',/SP_', '정', '종', '현/NNP_', ',/SP_', '신', '승', '일/NNP_', './SF_', '특례/NNG_', '시/NNB_', '지정/NNG_', '을/JKO_', '앞두/VV_', 'ㄴ/ETM_', '105/SN_', '만/NR_', '고양', '시/NNP_', '의/JKG_', '발전/NNG_', '을/JKO_', '위하/VV_', '어/EC_', '땀/NNG_', '과/JC_', '열정/NNG_', '을/JKO_', '쏟/VV_', '아/EC_', '오/VX_', 'ㄴ/ETM_', '6/SN_', '명/NNB_', '의/JKG_', '공직/NNG_', '자/XSN_', '(/SS_', '서기/NNG_', '관/XSN_', ')/SS_', '들/XSN_', '이/JKS_', '아름/NNG_', '답/XSA_', 'ㄴ/ETM_', '퇴장/NNG_', '을/JKO_', '앞두/VV_', '고/EC_', '있/VX_', '다/EF_', './SF_', '경기/NNP_', '북부/NNG_', '의/JKG_', '중심/NNG_', '도시/NNG_', '이/VCP_', 'ㄴ/ETM_', '고양', '시/NNP_', '가/JKC_', '되/VV_', '기/ETN_', '까지/JX_', '쉼', '/VV_', '없이/MAG_', '달려오/VV_', '았/EP_', '던/ETM_', '이/NP_', '들/XSN_', '은/JX_', '이제/MAG_', '후배/NNG_', '들/XSN_', '에게/JKB_', '자리/NNG_', '를/JKO_', '물', '려주/VV_', '고/EC_', '인생/NNG_', '2/SN_', '막', '/

In [ ]:
def convert_to_unicode(text):
    # Python version이 3.x일 때,
    # type(text)이 `bytes`일 경우, utf-8로 변환
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    # Python version이 2.x일 때,
    # type(text)이 `str`일 경우, utf-8로 변환
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    # Python 3.x, 2.x만 허용!
    else:
        raise ValueError("Not running on Python2 or Python 3?")
        
        
def _load_vocab(vocab_file):
    # 단어 사전을 저장할 OrderedDict 객체 생성
    vocab = collections.OrderedDict()
    index = 0
    with tf.io.gfile.GFile(vocab_file, 'r') as reader:
        while True:
            # Binary Text를 unicode(utf-8)로 decode.
            token = convert_to_unicode(reader.readline())
            if not token: break
            if ((token.find('n_iters=') == 0) or
                (token.find('max_length=') == 0)):
                continue
            token = token.split('\t')[0]
            token = token.strip()
            # 토큰과 해당 index를 기록
            vocab[token] = index
            index += 1
    return vocab

In [ ]:
import numpy as np

In [ ]:
df = df.iloc[:,1:]
df.head()

,media,a_num,title,content,w_num,c_source,a_source,s_num,v_num,nf_verb,s_verb,reporter,non_text,s_rate,quality,content1,depend
0,경기일보,329412764,"105만 고양시 발전 이끌어온 서기관 6명, 아름다운 퇴장","송주현 기자 왼쪽 위부터)김운용, 이현옥, 이흥민, 노양호, 정종현, 신승일 \r\...",1176,6,1.0,18,16,0,8,1,0,50.0,1,"송주현 기자 왼쪽 위부터)김운용, 이현옥, 이흥민, 노양호, 정종현, 신승일. 특례...",지정을 발전을 위해 열정을 공직자(서기관)들이 퇴장을 앞두고 있다. 되기까지 이들은...
1,경기일보,329412768,고양시 새해에는 교통난 해결될까 관심,송주현 기자 \r\n고양시민들이 출·퇴근길 겪고 있는 교통 불편이 새해에는 개선될 ...,1197,3,1.0,14,16,0,6,1,0,37.5,1,송주현 기자. 고양시민들이 출퇴근길 겪고 있는 교통 불편이 새해에는 개선될 수 있을...,고양시민들이 출퇴근길 겪고 불편이 수 있을지 관심이 모아진다. 따르면 구간)은 4량...
2,경기일보,329412771,"안양시, 오는 2022년까지 일자리 10만6천개 창출 목표",박준상 기자 \r\n기해년을 맞은 안양시가 향후 4년간 일자리 10만6천여개 창출을...,884,2,0.0,9,10,0,0,1,0,0.0,1,박준상 기자. 기해년을 맞은 안양시가 향후 4년간 일자리 10만6천여개 창출을 목표...,기해년을 안양시가 창출을 종합계획을 공시했다. 따르면 종합계획은 일자리를 전략과제를...
3,경기일보,329412773,"오명근 의원, 평택 고평지구 학군조정을 위한 의견수렴 간담회 개최",최현호 기자 \r\n경기도의회 건설교통위원회 소속 오명근 의원(더불어민주당ㆍ평택4)...,759,9,3.0,9,10,0,2,1,0,20.0,1,최현호 기자. 경기도의회 건설교통위원회 소속 오명근 의원(더불어민주당평택4)은 지난...,의원(더불어민주당평택4)은 참석해 대해 논의했다. 일원)는 평택교육청은 증축을 통해...
4,경기일보,329412774,"화성시, 2020년까지 신규 일자리 13만개 창출...일자리 대책 청사진 공개",홍완식 기자 \r\n화성시가 새해를 맞아 민선7기 지역맞춤형 일자리 대책의 청사진을...,937,2,0.0,8,8,0,0,1,0,0.0,1,홍완식 기자. 화성시가 새해를 맞아 민선7기 지역맞춤형 일자리 대책의 청사진을 공개...,화성시가 새해를 맞아 청사진을 공개했다. 시는 전산망을 통해 70를 종합계획을 공시...


In [ ]:
from tqdm import tqdm

In [ ]:
morph_vocab = _load_vocab(vocab_file)

In [ ]:
depend_list = ['SBJ', 'OBJ']

def do_lang2 ( openapi_key, text ) :
    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    
    requestJson = { "access_key": openapi_key, "argument": { "text": text, "analysis_code": "dparse" } }
    
    http = urllib3.PoolManager()
    response = http.request( "POST", openApiURL, headers={"Content-Type": "application/json; charset=UTF-8"}, body=json.dumps(requestJson))
    
    json_data = json.loads(response.data.decode('utf-8'))
    json_result = json_data["result"]
    
    if json_result == -1:
        json_reason = json_data["reason"]
        if "Invalid Access Key" in json_reason:
            logger.info(json_reason)
            logger.info("Please check the openapi access key.")
            sys.exit()
        return "openapi error - " + json_reason      
    else:
        json_data = json.loads(response.data.decode('utf-8'))
    
        json_return_obj = json_data["return_object"]
        
        return_result = ""
        ls_return = []
        sentence = json_return_obj['sentence']
        for morp in sentence:
          for dep in morp['dependency']:
            if dep['label'] == 'VP':
              return_result = return_result+str(dep["text"]) + ' '#+"/"+str(dep["label"])+" "
              #ls_return.append(str(dep["text"])+"/"+str(dep["label"]))
            for depend in depend_list:
              if depend in dep['label']:
                return_result = return_result+str(dep["text"]) + ' '#+"/"+str(dep["label"])+" "
                #ls_return.append(str(dep["text"])+"/"+str(dep["label"]))
          


        return return_result

In [ ]:
df['depend'] = df['content1'].apply(lambda x: do_lang2(api_key, x))

KeyboardInterrupt: ignored

In [ ]:
df.head()

,media,a_num,title,content,w_num,c_source,a_source,s_num,v_num,nf_verb,s_verb,reporter,non_text,s_rate,quality,content1,depend
0,경기일보,329412764,"105만 고양시 발전 이끌어온 서기관 6명, 아름다운 퇴장","송주현 기자 왼쪽 위부터)김운용, 이현옥, 이흥민, 노양호, 정종현, 신승일 \r\...",1176,6,1.0,18,16,0,8,1,0,50.0,1,"송주현 기자 왼쪽 위부터)김운용, 이현옥, 이흥민, 노양호, 정종현, 신승일. 특례...",지정을 발전을 위해 열정을 공직자(서기관)들이 퇴장을 앞두고 있다. 되기까지 이들은...
1,경기일보,329412768,고양시 새해에는 교통난 해결될까 관심,송주현 기자 \r\n고양시민들이 출·퇴근길 겪고 있는 교통 불편이 새해에는 개선될 ...,1197,3,1.0,14,16,0,6,1,0,37.5,1,송주현 기자. 고양시민들이 출퇴근길 겪고 있는 교통 불편이 새해에는 개선될 수 있을...,고양시민들이 출퇴근길 겪고 불편이 수 있을지 관심이 모아진다. 따르면 구간)은 4량...
2,경기일보,329412771,"안양시, 오는 2022년까지 일자리 10만6천개 창출 목표",박준상 기자 \r\n기해년을 맞은 안양시가 향후 4년간 일자리 10만6천여개 창출을...,884,2,0.0,9,10,0,0,1,0,0.0,1,박준상 기자. 기해년을 맞은 안양시가 향후 4년간 일자리 10만6천여개 창출을 목표...,기해년을 안양시가 창출을 종합계획을 공시했다. 따르면 종합계획은 일자리를 전략과제를...
3,경기일보,329412773,"오명근 의원, 평택 고평지구 학군조정을 위한 의견수렴 간담회 개최",최현호 기자 \r\n경기도의회 건설교통위원회 소속 오명근 의원(더불어민주당ㆍ평택4)...,759,9,3.0,9,10,0,2,1,0,20.0,1,최현호 기자. 경기도의회 건설교통위원회 소속 오명근 의원(더불어민주당평택4)은 지난...,의원(더불어민주당평택4)은 참석해 대해 논의했다. 일원)는 평택교육청은 증축을 통해...
4,경기일보,329412774,"화성시, 2020년까지 신규 일자리 13만개 창출...일자리 대책 청사진 공개",홍완식 기자 \r\n화성시가 새해를 맞아 민선7기 지역맞춤형 일자리 대책의 청사진을...,937,2,0.0,8,8,0,0,1,0,0.0,1,홍완식 기자. 화성시가 새해를 맞아 민선7기 지역맞춤형 일자리 대책의 청사진을 공개...,화성시가 새해를 맞아 청사진을 공개했다. 시는 전산망을 통해 70를 종합계획을 공시...


In [ ]:
#df.to_excel('df_n_v.xlsx', index=False)

In [ ]:
from tokenization_morp import FullTokenizer

# FullTokenizer
ftk = FullTokenizer(vocab_file, do_lower_case=False)

In [ ]:
df['depend']

0       지정을 발전을 위해 열정을 공직자(서기관)들이 퇴장을 앞두고 있다. 되기까지 이들은...
1       고양시민들이 출퇴근길 겪고 불편이 수 있을지 관심이 모아진다. 따르면 구간)은 4량...
2       기해년을 안양시가 창출을 종합계획을 공시했다. 따르면 종합계획은 일자리를 전략과제를...
3       의원(더불어민주당평택4)은 참석해 대해 논의했다. 일원)는 평택교육청은 증축을 통해...
4       화성시가 새해를 맞아 청사진을 공개했다. 시는 전산망을 통해 70를 종합계획을 공시...
                              ...                        
998     단행 앞두고 관심이 쏠리고 있다. 자리가 채워지기 따르면 관심이 것은 자리가 늘어나...
999     혜택을 기틀을 마련하겠습니다. 이용걸(57)회장은 의정부지회가 창립하며 안정을 위해...
1000    검찰이 귤을 훔치려다 치료감호를 청구했지만, 배심원들은 기각을 평결했다. 부장판사)...
1001    사리장마 겹쳐 우려돼 공사가 통행이 차단된다. 인천경제자유구역청은 차가 장마철을 맞...
1002    참석자들이 기념촬영을 하고 있다. 발전협의회는 토론회를 열었다. 옹진군수를 명이 참...
Name: depend, Length: 1003, dtype: object

In [ ]:
lsls = []
for text in df['depend']:
  ht = do_lang(api_key, text)
  lsls.append(ht)

KeyboardInterrupt: ignored

In [ ]:
lsls[0]

'지정/NNG 을/JKO 발전/NNG 을/JKO 위하/VV 어/EC 열정/NNG 을/JKO 공직/NNG 자/XSN (/SS 서기/NNG 관/XSN )/SS 들/XSN 이/JKS 퇴장/NNG 을/JKO 앞두/VV 고/EC 있/VX 다/EF ./SF 되/VV 기/ETN 까지/JX 이/NP 들/XSN 은/JX 자리/NNG 를/JKO 물려주/VV 고/EC 2/SN 막/NNB 을/JKO 푸른/NNG 도시/NNG 사업/NNG 소장/NNG 은/JX 입문/NNG 하/XSV 어/EC 고양시/NNP 가/JKS 자연/NNG 을/ETM 수/NNB 있/VA 도록/EC 노력/NNG 을/JKO 기울이/VV 어/EC 오/VX 았/EP 다/EF ./SF 그/NP 는/JX 산림/NNG 보호/NNG 유공/NNG 을/JKO 인정/NNG 받/XSV 아/EC 산림청장/NNG 표창/NNG 을/JKO 비롯하/VV 어/EC ,/SP 표창/NNG 들/XSN 이/JKS 충실/NNG 하/XSA 어/EC 공직/NNG 생활/NNG 을/JKO 보이/VV 어/EC 주/VX ㄴ다/EF ./SF 고양시/NNP 를/JKO 교육/NNG 문화/NNG 국장/NNG 도/JX 공직/NNG 을/JKO 마무리/NNG 하/XSV 기/ETN 위하/VV 어/EC 들어가/VV ㄴ다/EF ./SF 부드럽/VA 고/EC 업무/NNG 스타일/NNG 은/JX 박수/NNG 를/JKO 받/VV 아/EC 오/VX 았/EP 다/EF ./SF 표창/NNG 을/JKO 13/SN 회/NNB 나/JC 민생/NNG 경제/NNG 국장/NNG 그/NP 는/JX 주민/NNG 들/XSN 이/JKS 생활/NNG 지원/NNG 을/XSV 수/NNB 있/VA 도록/EC 함께/MAG 하/XSV 며/EC 목/NNG 소리/NNG 를/JKO 담아오/VV 았/EP 다/EF ./SF 여성/NNG 가족/NNG 국장/NNG 도/JX 발전/NNG 을/JKO 위하/VV 어/EC 40/SN 년/NNB 넘/VV 게/EC 활약/NNG 하/XSV 어/EC 오/VX 았/EP 다/EF ./SF 개선/NNG

In [ ]:
lsls_tok = []
for text in lsls:
  split_tokens = ftk.tokenize(text)
  lsls_tok.append(split_tokens)

In [ ]:
mmm = 0
for tok in lsls_tok:
  if len(tok) > mmm:
    mmm = len(tok)

mmm

607

In [ ]:
print(lsls_tok)

[['지정/NNG_', '을/JKO_', '발전/NNG_', '을/JKO_', '위하/VV_', '어/EC_', '열정/NNG_', '을/JKO_', '공직/NNG_', '자/XSN_', '(/SS_', '서기/NNG_', '관/XSN_', ')/SS_', '들/XSN_', '이/JKS_', '퇴장/NNG_', '을/JKO_', '앞두/VV_', '고/EC_', '있/VX_', '다/EF_', './SF_', '되/VV_', '기/ETN_', '까지/JX_', '이/NP_', '들/XSN_', '은/JX_', '자리/NNG_', '를/JKO_', '물', '려주/VV_', '고/EC_', '2/SN_', '막', '/NNB_', '을/JKO_', '푸', '른/NNG_', '도시/NNG_', '사업/NNG_', '소장/NNG_', '은/JX_', '입', '문/NNG_', '하/XSV_', '어/EC_', '고양', '시/NNP_', '가/JKS_', '자연/NNG_', '을/ETM_', '수/NNB_', '있/VA_', '도록/EC_', '노력/NNG_', '을/JKO_', '기울이/VV_', '어/EC_', '오/VX_', '았/EP_', '다/EF_', './SF_', '그/NP_', '는/JX_', '산', '림/NNG_', '보호/NNG_', '유', '공/NNG_', '을/JKO_', '인정/NNG_', '받/XSV_', '아/EC_', '산', '림', '청장/NNG_', '표', '창/NNG_', '을/JKO_', '비롯하/VV_', '어/EC_', ',/SP_', '표', '창/NNG_', '들/XSN_', '이/JKS_', '충실/NNG_', '하/XSA_', '어/EC_', '공직/NNG_', '생활/NNG_', '을/JKO_', '보이/VV_', '어/EC_', '주/VX_', 'ㄴ다/EF_', './SF_', '고양', '시/NNP_', '를/JKO_', '교육/NNG_', '문화/NNG_', '국장/NNG_', '도/JX_', '공직/

In [ ]:
import numpy as np
from tqdm import tqdm

morph_vocab = _load_vocab(vocab_file)

def toktok_n_v(df):


  maxlen = 512

  ipt_ids = []

  ipt_mask = []
   
  sgm_ids = []

  #df['depend'] = df['content1'].apply(lambda x: do_lang2(api_key, x))





  
  for content in tqdm(df['depend']):

    text = do_lang('f3639d57-7635-4bd9-93b0-6eae550ab90e', content)
    split_tokens = ftk.tokenize(text)

    tokens = ["[CLS]"] + split_tokens[:510] + ["[SEP]"]

    input_ids = [morph_vocab[token] for token in tokens]
    
    input_mask = [1] * len(input_ids)

    segment_ids = [0] * len(tokens)
    
    padding = [0] * (maxlen - len(input_ids))
    
    input_ids += padding
    
    input_mask += padding
    
    segment_ids += padding
    
    ipt_ids.append(input_ids) # 토큰
    
    ipt_mask.append(input_mask)
    

    sgm_ids.append(segment_ids) # 문장 분류
    

  ar_tok = np.array(ipt_ids)
  
  ar_seg = np.array(sgm_ids)
    
  train_x = [[ar_tok[:900]], [ar_seg[:900]]]

  test_x = [[ar_tok[900:]], [ar_seg[900:]]]

  y = df['quality']
  train_y = np.array(y[:900])
  test_y = np.array(y[900:])

  return train_x, test_x, train_y, test_y

In [ ]:
train_x, test_x, train_y, test_y = toktok_n_v(df)

100%|██████████| 1003/1003 [11:08<00:00,  1.50it/s]


In [ ]:
!pip install keras-bert

In [ ]:
#!pip install keras-bert

from keras.models import Model
from keras import layers
from keras import Input
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
#from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
#from transformers import *

#from keras_radam import RAdam


config_path = '/content/gdrive/My Drive/KorBERT/bert_config.json'
checkpoint_path = '/content/gdrive/My Drive/KorBERT/model.ckpt'
SEQ_LEN = 512

layer_num = 12

model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)
#model = TFBertForPreTraining.from_pretrained(checkpoint_path)
#model = TFBertModel.from_pretrained(checkpoint_path, from_pt=True)
#token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
#mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
#segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
#bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

In [ ]:
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output


  outputs = keras.layers.Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)



  bert_model = keras.models.Model(inputs, outputs)
  bert_model.compile(
      optimizer=tf.keras.optimizers.Adam(),
      loss='binary_crossentropy',
      metrics=['accuracy'])
  
  return bert_model

In [ ]:
import keras

In [ ]:
bert_model = get_bert_finetuning_model(model)
history = bert_model.fit([train_x[0],train_x[1]], train_y, epochs=3, batch_size=4, verbose = 1, validation_split=0.1)

Epoch 1/3
 29/203 [===>..........................] - ETA: 1:09:24 - loss: 0.7765 - accuracy: 0.5000

In [ ]:
df.head()

,Unnamed: 0,media,a_num,title,content,w_num,c_source,a_source,s_num,v_num,nf_verb,s_verb,reporter,non_text,s_rate,quality,content1,depend
0,0,경기일보,329412764,"105만 고양시 발전 이끌어온 서기관 6명, 아름다운 퇴장","송주현 기자 왼쪽 위부터)김운용, 이현옥, 이흥민, 노양호, 정종현, 신승일 \r\...",1176,6,1.0,18,16,0,8,1,0,50.0,1,"송주현 기자 왼쪽 위부터)김운용, 이현옥, 이흥민, 노양호, 정종현, 신승일. 특례...",지정을 발전을 위해 열정을 공직자(서기관)들이 퇴장을 앞두고 있다. 되기까지 이들은...
1,1,경기일보,329412768,고양시 새해에는 교통난 해결될까 관심,송주현 기자 \r\n고양시민들이 출·퇴근길 겪고 있는 교통 불편이 새해에는 개선될 ...,1197,3,1.0,14,16,0,6,1,0,37.5,1,송주현 기자. 고양시민들이 출퇴근길 겪고 있는 교통 불편이 새해에는 개선될 수 있을...,고양시민들이 출퇴근길 겪고 불편이 수 있을지 관심이 모아진다. 따르면 구간)은 4량...
2,2,경기일보,329412771,"안양시, 오는 2022년까지 일자리 10만6천개 창출 목표",박준상 기자 \r\n기해년을 맞은 안양시가 향후 4년간 일자리 10만6천여개 창출을...,884,2,0.0,9,10,0,0,1,0,0.0,1,박준상 기자. 기해년을 맞은 안양시가 향후 4년간 일자리 10만6천여개 창출을 목표...,기해년을 안양시가 창출을 종합계획을 공시했다. 따르면 종합계획은 일자리를 전략과제를...
3,3,경기일보,329412773,"오명근 의원, 평택 고평지구 학군조정을 위한 의견수렴 간담회 개최",최현호 기자 \r\n경기도의회 건설교통위원회 소속 오명근 의원(더불어민주당ㆍ평택4)...,759,9,3.0,9,10,0,2,1,0,20.0,1,최현호 기자. 경기도의회 건설교통위원회 소속 오명근 의원(더불어민주당평택4)은 지난...,의원(더불어민주당평택4)은 참석해 대해 논의했다. 일원)는 평택교육청은 증축을 통해...
4,4,경기일보,329412774,"화성시, 2020년까지 신규 일자리 13만개 창출...일자리 대책 청사진 공개",홍완식 기자 \r\n화성시가 새해를 맞아 민선7기 지역맞춤형 일자리 대책의 청사진을...,937,2,0.0,8,8,0,0,1,0,0.0,1,홍완식 기자. 화성시가 새해를 맞아 민선7기 지역맞춤형 일자리 대책의 청사진을 공개...,화성시가 새해를 맞아 청사진을 공개했다. 시는 전산망을 통해 70를 종합계획을 공시...
